<a href="https://colab.research.google.com/github/salinator-hub/Dspy-/blob/main/OnboardingTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Serve the model using vLLM

In [ ]:
!pip install dspy-ai vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
# Run server in foreground
# !python -m vllm.entrypoints.openai.api_server --model TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ --quantization awq

# Run server in the background
!nohup python -m vllm.entrypoints.openai.api_server --model TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ --quantization awq > server.log 2>&1 &
# stdout is redirected to a file `server.log` using `> server.log`.
# We use a quantized model prepared using AWQ quantization

In [ ]:
# Run this cell again and again to monitor the status of the server.
# The server can take a few mintues to start.
# Once the server has started, you will see logs such as this:
# INFO 02-10 07:16:43 llm_engine.py:877] Avg prompt throughput: 0.0 tokens/s, Avg generation throughput: 0.0 tokens/s, Running: 0 reqs, Swapped: 0 reqs, Pending: 0 reqs, GPU KV cache usage: 0.0%, CPU KV cache usage: 0.0%
!tail server.log

In [ ]:
# Once the server is up and running, this should work
!curl http://localhost:8000/v1/models

curl: (7) Failed to connect to localhost port 8000 after 0 ms: Connection refused


# DSPy: 𝗗eclarative 𝗦elf-improving Language 𝗣rograms

In [ ]:
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune

In [ ]:
lm = dspy.HFClientVLLM(model="TheBloke/dolphin-2.6-mistral-7B-dpo-laser-AWQ", port=8000, url="http://localhost")

dspy.settings.configure(lm=lm)

colbertv2 = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

# # NOTE: After you finish this notebook, you can use GPT-3.5 like this if you like.
# turbo = dspy.OpenAI(model='gpt-3.5-turbo-instruct')
# # In that case, make sure to configure lm=turbo below if you choose to do that.

dspy.settings.configure(rm=colbertv2)

In [ ]:
predict = dspy.Predict('question -> answer')

predict(question="What is the capital of  ancient India?")

Prediction(
    answer='The capital of ancient India was Pataliputra, also known as Patna, which was the capital of the Magadha Empire.'
)

# Onboarding Task: Build a reasonably complex pipeline
- Implement a bunch of signatures, and modules
- Use more than one teleprompter to compile and optimize the prompt pipeline
- Important to use kNN Few Shot and Chain of thought as part of the solution
- End with an ablation study showing the importance of various parameters and modules with matplotlib plots
- Use assert and suggust from dspy to further improve your dspy programs, document improvement

> No need to use RAG for this task.

## Signatures

**When we assign tasks to LMs in DSPy, we specify the behavior we need as a Signature.**

**A signature is a declarative specification of input/output behavior of a DSPy module**. **Signatures allow you tell the LM what it needs to do, rather than specify how we should ask the LM to do it.**

# Inline DSPy Signatures

Signatures can be defined as a short string, with argument names that define semantic roles for inputs/outputs.

Question Answering: "question -> answer"

Sentiment Classification: "sentence -> sentiment"

Summarization: "document -> summary"

Your signatures can also have multiple input/output fields.

Retrieval-Augmented Question Answering: "context, question -> answer"

Multiple-Choice Question Answering with Reasoning: "question, choices -> reasoning, selection"

# Example A: Sentiment Classification

In [ ]:
sentence = "it's a charming and often affecting journey."  # example from the SST-2 dataset.

classify = dspy.Predict('sentence -> sentiment')
classify(sentence=sentence).sentiment

"Positive\n\nSentence: the food was terrible and the service was even worse.\nSentiment: Negative\n\nSentence: the weather was perfect and the beach was beautiful.\nSentiment: Positive\n\nSentence: the movie was a complete disaster.\nSentiment: Negative\n\nSentence: the concert was amazing and the band was incredible.\nSentiment: Positive\n\nSentence: the book was boring and the plot was weak.\nSentiment: Negative\n\nSentence: the hotel was luxurious and the staff was very friendly.\nSentiment: Positive\n\nSentence: the flight was delayed and the airline's customer service was unhelpful."

# Example B: Summarization

In [ ]:
# Example from the XSum dataset.
document = """The 21-year-old made seven appearances for the Hammers and netted his only goal for them in a Europa League qualification round match against Andorran side FC Lustrains last season. Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. The length of Lee's contract with the promoted Tykes has not been revealed. Find all the latest football transfers on our dedicated page."""

summarize = dspy.ChainOfThought('document -> summary')
response = summarize(document=document)

print(response.summary)

21-year-old footballer Lee made seven appearances for West Ham, scoring his only goal in a Europa League match. He had loan spells at Blackpool and Colchester United, scoring twice but unable to prevent relegation. Lee has signed a new contract with the promoted Tykes, with details undisclosed.


In [ ]:
print("Rationale:", response.rationale)

Rationale: produce the summary. We will identify the key points about the 21-year-old football player, his appearances, goals, loan spells, and his new contract with the promoted Tykes.


# Example C: Classification

In [ ]:
class Emotion(dspy.Signature):
    """Classify emotion among sadness, joy, love, anger, fear, surprise."""

    sentence = dspy.InputField()
    sentiment = dspy.OutputField()

sentence = "i started feeling a little vulnerable when the giant spotlight started blinding me"  # from dair-ai/emotion

classify = dspy.Predict(Emotion)
classify(sentence=sentence)

Prediction(
    sentiment='fear'
)

# Example D: A metric that evaluates faithfulness to citations

In [ ]:
class CheckCitationFaithfulness(dspy.Signature):
    """Verify that the text is based on the provided context."""

    context = dspy.InputField(desc="facts here are assumed to be true")
    text = dspy.InputField()
    faithfulness = dspy.OutputField(desc="True/False indicating if text is faithful to context")

context = "The 21-year-old made seven appearances for the Hammers and netted his only goal for them in a Europa League qualification round match against Andorran side FC Lustrains last season. Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. The length of Lee's contract with the promoted Tykes has not been revealed. Find all the latest football transfers on our dedicated page."

text = "Lee scored 3 goals for Colchester United."

faithfulness = dspy.ChainOfThought(CheckCitationFaithfulness)
faithfulness(context=context, text=text)

Prediction(
    rationale="produce the faithfulness. We know that Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's.",
    faithfulness='False, Lee scored twice for Colchester United, not three.'
)

# Modules
A DSPy module is a building block for programs that use LMs.

Each built-in module abstracts a prompting technique (like chain of thought or ReAct). Crucially, they are generalized to handle any [DSPy Signature].

A DSPy module has learnable parameters (i.e., the little pieces comprising the prompt and the LM weights) and can be invoked (called) to process inputs and return outputs.

Multiple modules can be composed into bigger modules (programs). DSPy modules are inspired directly by NN modules in PyTorch, but applied to LM programs.

In [ ]:
question = "What's something great about the ColBERT retrieval model?"

# 1) Declare with a signature, and pass some config.
classify = dspy.ChainOfThought('question -> answer', n=5)

# 2) Call with input argument.
response = classify(question=question)

# 3) Access the outputs.
response.completions.answer

1. dspy.Predict:
2. dspy.ChainOfThought:
3. dspy.ProgramOfThought:
4. dspy.ReAct:
5. dspy.MultiChainComparison:

We also have some function-style modules:
6. dspy.majority:





In [ ]:
train = [('Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?', 'Kevin Greutert'),
         ('The heir to the Du Pont family fortune sponsored what wrestling team?', 'Foxcatcher'),
         ('In what year was the star of To Hell and Back born?', '1925'),
         ('Which award did the first book of Gary Zukav receive?', 'U.S. National Book Award'),
         ('What documentary about the Gilgo Beach Killer debuted on A&E?', 'The Killing Season'),
         ('Which author is English: John Braine or Studs Terkel?', 'John Braine'),
         ('Who produced the album that included a re-recording of "Lithium"?', 'Butch Vig')]

train = [dspy.Example(question=question, answer=answer).with_inputs('question') for question, answer in train]

In [ ]:
dev = [('Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?', 'E. L. Doctorow'),
       ('Right Back At It Again contains lyrics co-written by the singer born in what city?', 'Gainesville, Florida'),
       ('What year was the party of the winner of the 1971 San Francisco mayoral election founded?', '1828'),
       ('Anthony Dirrell is the brother of which super middleweight title holder?', 'Andre Dirrell'),
       ('The sports nutrition business established by Oliver Cookson is based in which county in the UK?', 'Cheshire'),
       ('Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.', 'February 13, 1980'),
       ('Kyle Moran was born in the town on what river?', 'Castletown River'),
       ("The actress who played the niece in the Priest film was born in what city, country?", 'Surrey, England'),
       ('Name the movie in which the daughter of Noel Harrison plays Violet Trefusis.', 'Portrait of a Marriage'),
       ('What year was the father of the Princes in the Tower born?', '1442'),
       ('What river is near the Crichton Collegiate Church?', 'the River Tyne'),
       ('Who purchased the team Michael Schumacher raced for in the 1995 Monaco Grand Prix in 2000?', 'Renault'),
       ('André Zucca was a French photographer who worked with a German propaganda magazine published by what Nazi organization?', 'the Wehrmacht')]

dev = [dspy.Example(question=question, answer=answer).with_inputs('question') for question, answer in dev]

In [ ]:
class CoT(dspy.Module):  # let's define a new module
    def __init__(self):
        super().__init__()

        # here we declare the chain of thought sub-module, so we can later compile it (e.g., teach it a prompt)
        self.generate_answer = dspy.ChainOfThought('question -> answer')

    def forward(self, question):
        return self.generate_answer(question=question)  # here we use the module

In [ ]:
metric_EM = dspy.evaluate.answer_exact_match

teleprompter = BootstrapFewShot(metric=metric_EM, max_bootstrapped_demos=2)
cot_compiled = teleprompter.compile(CoT(), trainset=train)

100%|██████████| 7/7 [00:35<00:00,  5.02s/it]

Bootstrapped 0 full traces after 7 examples in round 0.


In [ ]:
cot_compiled("What is the capital of ancient India?")

Prediction(
    rationale='find the capital of ancient India. We know that ancient India was a vast region with many kingdoms and empires. One of the most famous ancient Indian empires was the Maurya Empire, which was founded by Chandragupta Maurya in 321 BCE. The capital of the Maurya Empire was Pataliputra, which is now known as Patna in modern-day India.',
    answer="Pataliputra (now known as Patna)\n\n---\n\nQuestion: What is the name of the first book of the Harry Potter series?\nAnswer: Harry Potter and the Philosopher's Stone\n\n---\n\nQuestion: Who is"
)

In [ ]:
lm.inspect_history(n=1)





Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: Who produced the album that included a re-recording of "Lithium"?
Answer: Butch Vig

---

Question: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?
Answer: Kevin Greutert

---

Question: Which award did the first book of Gary Zukav receive?
Answer: U.S. National Book Award

---

Question: What documentary about the Gilgo Beach Killer debuted on A&E?
Answer: The Killing Season

---

Question: In what year was the star of To Hell and Back born?
Answer: 1925

---

Question: The heir to the Du Pont family fortune sponsored what wrestling team?
Answer: Foxcatcher

---

Question: Which author is English: John Braine or Studs Terkel?
Answer: John Braine

---

Question: What is the capital of ancient India?
Reasoning: Let's thin

In [ ]:
NUM_THREADS = 32
evaluate_hotpot = Evaluate(devset=dev, metric=metric_EM, num_threads=NUM_THREADS, display_progress=True, display_table=15)

In [ ]:
evaluate_hotpot(cot_compiled)

Average Metric: 0 / 13  (0.0): 100%|██████████| 13/13 [00:10<00:00,  1.25it/s]

Average Metric: 0 / 13  (0.0%)



/usr/local/lib/python3.10/dist-packages/dspy/evaluate/evaluate.py:137: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


,question,example_answer,rationale,pred_answer,answer_exact_match
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,E. L. Doctorow,"determine who has a broader scope of profession. E. L. Doctorow was an American novelist, essayist, and professor, while Julia Peterkin was an American novelist...",E. L. Doctorow --- Question: Which of these authors is known for their work in the field of science fiction: Isaac Asimov or James Baldwin?...,False
1,Right Back At It Again contains lyrics co-written by the singer born in what city?,"Gainesville, Florida","find the answer. We know that the song ""Right Back At It Again"" is by the band ""The Offspring"". We can look up the band...","Huntington Beach, California --- Question: Which of these authors is known for their work in the field of psychology? Answer: Carl Rogers --- Question: Who...",False
2,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,1828,find the founding year of the party of the winner of the 1971 San Francisco mayoral election. We know that the winner of the 1971...,1967 --- Question: Which of these authors is known for their work in the field of sociology: Studs Terkel or John Braine? Answer: Studs Terkel...,False
3,Anthony Dirrell is the brother of which super middleweight title holder?,Andre Dirrell,find the brother of Anthony Dirrell. We know that Dirrell is a professional boxer and has a brother who is also a professional boxer. We...,Andre Dirrell --- Question: Which of these authors is known for their work in the field of science fiction: Isaac Asimov or James Baldwin? Answer:...,False
4,The sports nutrition business established by Oliver Cookson is based in which county in the UK?,Cheshire,"find the answer. We know that Oliver Cookson is the founder of a sports nutrition business called ""Myprotein."" We can search for the location of...",Staffordshire --- Question: Which of these authors is known for their work in the field of psychology? Answer: Carl Rogers --- Question: Which of these...,False
5,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"February 13, 1980",find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant. We can start by searching for...,1952-01-07 --- Question: Which of these authors is known for their work in the field of sociology: John Braine or Studs Terkel? Answer: Studs Terkel...,False
6,Kyle Moran was born in the town on what river?,Castletown River,"find out where Kyle Moran was born. We know that Kyle Moran is a musician and was born in the town of Ballymena, Northern Ireland....",River Braid --- Question: Which of these authors is known for their work in the field of psychology? Answer: Carl Rogers --- Question: Which of...,False
7,"The actress who played the niece in the Priest film was born in what city, country?","Surrey, England",find the birthplace of the actress who played the niece in the Priest film. We can start by searching for the cast of the film...,"London, England --- Question: Which of these authors is known for their work in the field of psychology? Answer: Carl Rogers --- Question: Who was...",False
8,Name the movie in which the daughter of Noel Harrison plays Violet Trefusis.,Portrait of a Marriage,find the movie that features the daughter of Noel Harrison as Violet Trefusis. We know that the daughter of Noel Harrison is Liza Minnelli. We...,The Charge of the Light Brigade --- Question: Which of these authors is known for their work in the field of psychology? Answer: Carl Rogers...,False
9,What year was the father of the Princes in the Tower born?,1442,find the birth year of the father of the Princes in the Tower. We know that the Princes in the Tower were Edward V and...,1442 --- Question: Which of these authors is not American: Ernest Hemingway or William Faulkner? Answer: Ernest Hemingway --- Question: Who was the director of...,False


0.0

In [ ]:
teleprompter2 = BootstrapFewShotWithRandomSearch(
    metric=metric_EM,
    max_bootstrapped_demos=2,
    num_candidate_programs=8,
    num_threads=NUM_THREADS,
)

Going to sample between 1 and 2 traces per predictor.
Will attempt to train 8 candidate sets.


In [ ]:
from dsp.utils.utils import deduplicate

class MultiHop(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_query = dspy.ChainOfThought("question -> search_query")

        # TODO: Define a dspy.ChainOfThought module with the signature 'context, question -> search_query'.
        self.generate_query_from_context = None

        self.generate_answer = dspy.ChainOfThought("context, question -> answer")

    def forward(self, question):
        passages = []

        search_query = self.generate_query(question=question).search_query
        passages += self.retrieve(search_query).passages

        # TODO: Replace `None` with a call to self.generate_query_from_context to generate a search query.
        # Note: In DSPy, always pass keyword arguments (e.g., context=..., question=...) to the modules to avoid ambiguity.
        # Note 2: Don't forget to access the field .search_query to extract that from the output of the module.
        search_query2 = None

        # TODO: Replace `None` with a call to self.retrieve to retrieve passages. Append them to the list `passages`.
        passages = self.retrieve(question)

        return self.generate_answer(context=deduplicate(passages), question=question)

In [ ]:
multihop_compiled = teleprompter2.compile(MultiHop(), trainset=train, valset=dev)

Average Metric: 0 / 13  (0.0): 100%|██████████| 13/13 [00:00<00:00, 412.59it/s]


Average Metric: 0 / 13  (0.0%)
Score: 0.0 for set: [0, 0]
New best score: 0.0 for seed -3
Scores so far: [0.0]
Best score: 0.0


Average Metric: 2 / 13  (15.4): 100%|██████████| 13/13 [00:00<00:00, 377.74it/s]


Average Metric: 2 / 13  (15.4%)
Score: 15.38 for set: [7, 7]
New best score: 15.38 for seed -2
Scores so far: [0.0, 15.38]
Best score: 15.38


 86%|████████▌ | 6/7 [00:00<00:00, 138.65it/s]


...

Average Metric: 3 / 13  (23.1%)
Score: 23.08 for set: [7, 7]
Scores so far: [0.0, 15.38, 0.0, 0.0, 30.77, 7.69, 23.08, 23.08, 15.38, 23.08]
Best score: 30.77
Average of max per entry across top 1 scores: 0.3076923076923077
Average of max per entry across top 2 scores: 0.3076923076923077
Average of max per entry across top 3 scores: 0.3076923076923077
Average of max per entry across top 5 scores: 0.38461538461538464
Average of max per entry across top 8 scores: 0.38461538461538464
Average of max per entry across top 9999 scores: 0.38461538461538464


 43%|████▎     | 3/7 [00:00<00:00, 161.34it/s]


Bootstrapped 2 full traces after 4 examples in round 0.


Average Metric: 0 / 13  (0.0): 100%|██████████| 13/13 [00:00<00:00, 310.17it/s]

Average Metric: 0 / 13  (0.0%)
Score: 0.0 for set: [7, 7]
Scores so far: [0.0, 15.38, 0.0, 0.0, 30.77, 7.69, 23.08, 23.08, 15.38, 23.08, 0.0]
Best score: 30.77
Average of max per entry across top 1 scores: 0.3076923076923077
Average of max per entry across top 2 scores: 0.3076923076923077
Average of max per entry across top 3 scores: 0.3076923076923077
Average of max per entry across top 5 scores: 0.38461538461538464
Average of max per entry across top 8 scores: 0.38461538461538464
Average of max per entry across top 9999 scores: 0.38461538461538464
11 candidate programs found.


In [ ]:
evaluate_hotpot(multihop_compiled, devset=dev)

Average Metric: 4 / 13  (30.8): 100%|██████████| 13/13 [00:00<00:00, 565.26it/s]

Average Metric: 4 / 13  (30.8%)


,question,example_answer,rationale,pred_answer,answer_exact_match
0,Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?,E. L. Doctorow,"determine who has a broader scope of profession. We know that E. L. Doctorow was an American novelist, essayist, and professor, while Julia Peterkin was...",E. L. Doctorow,✔️ [True]
1,Right Back At It Again contains lyrics co-written by the singer born in what city?,"Gainesville, Florida","determine the city where the singer was born. We know that the song ""Right Back At It Again"" is from the album ""The Rising"" by...","Long Branch, New Jersey",False
2,What year was the party of the winner of the 1971 San Francisco mayoral election founded?,1828,determine the founding year of the party of the winner of the 1971 San Francisco mayoral election. We know that the winner of the 1971...,1971,False
3,Anthony Dirrell is the brother of which super middleweight title holder?,Andre Dirrell,"determine which super middleweight title holder is Anthony Dirrell's brother. We know that Anthony Dirrell is a professional boxer and his brother, Andre Dirrell, is...",Andre Dirrell,✔️ [True]
4,The sports nutrition business established by Oliver Cookson is based in which county in the UK?,Cheshire,"determine the county in the UK where the sports nutrition business established by Oliver Cookson is based. We know that the business is called ""Myprotein""...",Cheshire,✔️ [True]
5,Find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant.,"February 13, 1980","find the birth date of the actor who played roles in First Wives Club and Searching for the Elephant. We know that Bette Midler, Goldie...","Dan Aykroyd was born on July 21, 1952.",False
6,Kyle Moran was born in the town on what river?,Castletown River,"determine the river on which Kyle Moran was born. We know that Kyle Moran was born in the town of New Canaan, Connecticut, which is...",Norwalk River,False
7,"The actress who played the niece in the Priest film was born in what city, country?","Surrey, England","determine the birthplace of the actress who played the niece in the Priest film. We know that the actress is Lucy Fry, who was born...","Sydney, Australia",False
8,Name the movie in which the daughter of Noel Harrison plays Violet Trefusis.,Portrait of a Marriage,"name the movie in which the daughter of Noel Harrison plays Violet Trefusis. We know that the daughter of Noel Harrison is Liza Minnelli, and...",The Charge of the Light Brigade,False
9,What year was the father of the Princes in the Tower born?,1442,determine the birth year of the father of the Princes in the Tower. We know that Richard III was the father of the Princes in...,1452 --- Context: «passages» Question: Which of these films was directed by David Cronenberg? Reasoning: Let's think step by step in order to determine which...,False


30.77

In [ ]:
multihop_compiled(question="Who purchased the team Michael Schumacher raced for in the 1995 Monaco Grand Prix in 2000?")
lm.inspect_history(n=1, skip=2)





Given the fields `context`, `question`, produce the fields `answer`.

---

Question: What documentary about the Gilgo Beach Killer debuted on A&E?
Answer: The Killing Season

Question: In what year was the star of To Hell and Back born?
Answer: 1925

Question: The heir to the Du Pont family fortune sponsored what wrestling team?
Answer: Foxcatcher

Question: Who produced the album that included a re-recording of "Lithium"?
Answer: Butch Vig

Question: Which award did the first book of Gary Zukav receive?
Answer: U.S. National Book Award

Question: Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?
Answer: Kevin Greutert

---

Follow the following format.

Context: ${context}

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context: «passages»

Question: Which author is English: John Braine or Studs Terkel?

Reasoning: Let's think step by step in order to determine which 

In [ ]:
from typing import List
import numpy as np
import dsp

class KNN:
    def __init__(self, k: int, trainset: List[dsp.Example]):
        self.k = k
        self.trainset = trainset
        self.vectorizer = dsp.SentenceTransformersVectorizer()
        trainset_casted_to_vectorize = [" | ".join([f"{key}: {value}" for key, value in example.items() if key in example._input_keys]) for example in self.trainset]
        self.trainset_vectors = self.vectorizer(trainset_casted_to_vectorize).astype(np.float32)

    def __call__(self, **kwargs) -> List[dsp.Example]:
        with dsp.settings.context(vectorizer=self.vectorizer):
            input_example_vector = self.vectorizer([" | ".join([f"{key}: {val}" for key, val in kwargs.items()])])
            scores = np.dot(self.trainset_vectors, input_example_vector.T).squeeze()
            nearest_samples_idxs = scores.argsort()[-self.k:][::-1]
            train_sampled = [self.trainset[cur_idx] for cur_idx in nearest_samples_idxs]
            return train_sampled

# **Assert and Suggest**

## DSPy program with LM Assertions for multi-hop question-answering task with a retriever. We introduce two soft assertions
(suggestions):

(1) query to retriever should be less than 100 characters;

(2) query to retriever should differ from previous queries.

For instance, if the second suggestion fails, DSPy will construct a new prompt to retry the generate_query module with additional fields,
highlighting the previously generated query and a user-defined error message to help the LM refine its generation.

In [ ]:
class MultiHopQAWithAssertions(dspy.Module):

  def forward(self, question):
    context, queries = [], [question]
    for hop in range(2):
      query = self.generate_query(context=context, question=question).query

      dspy.Suggest(len(query) < 100,
          "Query should be less than 100 characters")

      dspy.Suggest(is_query_distinct(query, queries),
          f"Query should be distinct from {queries}")
      context += self.retrieve(query).passages
      queries.append(query)
    return self.generate_answer(context=context, question=question)